In [1]:
import pandas as pd


from constants import okr_excel_path, researcher_info_excel_path, salesperson_info_excel_path
from utils import get_df_roadshow, get_sales_name_list, get_sales_name_list
from utils import print_na_rate

In [2]:
# # Get the Excel file
# xls = pd.ExcelFile(excel_path)

# # Get sheet names
# sheet_names = xls.sheet_names
# print(sheet_names)  # e.g., ['Sheet1', 'Sheet2', ...]

# # Close the Excel file
# xls.close()

In [3]:
df_okr = pd.read_excel(okr_excel_path, sheet_name='路演')
df_researcher_info = pd.read_excel(researcher_info_excel_path, usecols='A:C')
df_salespeople_info =  pd.read_excel(salesperson_info_excel_path)

In [4]:
sales_name_list = get_sales_name_list(df_salespeople_info)

In [5]:
df_roadshow, df_special = get_df_roadshow(df_okr, sales_name_list)

UnboundLocalError: cannot access local variable 'df_special' where it is not associated with a value

In [ ]:
df_roadshow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3449 entries, 0 to 3448
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   序号      3449 non-null   int64 
 1   服务事项    3449 non-null   object
 2   客户机构    3449 non-null   object
 3   客户分级    3449 non-null   object
 4   客户区域    3449 non-null   object
 5   所属团队    3449 non-null   object
 6   研究员     3449 non-null   object
dtypes: int64(1), object(6)
memory usage: 188.7+ KB


In [ ]:
print_na_rate(df_roadshow)

The na rate of the DataFrame is:
序号      0.0
服务事项    0.0
客户机构    0.0
客户分级    0.0
客户区域    0.0
所属团队    0.0
研究员     0.0
dtype: float64


In [ ]:
df_roadshow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3449 entries, 0 to 3448
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   序号      3449 non-null   int64 
 1   服务事项    3449 non-null   object
 2   客户机构    3449 non-null   object
 3   客户分级    3449 non-null   object
 4   客户区域    3449 non-null   object
 5   所属团队    3449 non-null   object
 6   研究员     3449 non-null   object
dtypes: int64(1), object(6)
memory usage: 188.7+ KB


In [ ]:
# save to excel with sheetname roadshow without index
df_roadshow.to_excel('okr_roadshow_202306.xlsx', sheet_name='roadshow', index=False)